In [ ]:
import numpy as np
import time
from Helper import LearningCurvePlot, smooth
from Assignment2_batch_updates import *

def average_over_repetitions(policy, epsilon, smoothing_window, plot, decay_epsilon, n_repetitions, n_runs, learning_rate,
                             gamma, batch_size, experience_replay):

    reward_results = np.empty([n_repetitions,n_runs]) # Result array
    now = time.time()
    
    for rep in range(n_repetitions): # Loop over repetitions
        rewards = cartpole(n_runs, learning_rate, gamma, policy, epsilon, experience_replay, batch_size, decay_epsilon)
        reward_results[rep] = rewards
        
    print('Running one setting takes {} minutes'.format((time.time()-now)/60))    
    learning_curve = np.mean(reward_results,axis=0) # average over repetitions
    #learning_curve = smooth(learning_curve,smoothing_window) # additional smoothing
    return learning_curve  

def experiment():
    ####### Settings
    # Experiment    
    n_repetitions = 8
    smoothing_window = 1001
    n_runs = 100
    experience_replay = True
    gamma = 0.995
    optimal_average_reward_per_timestep = 300

    # Exploration
    policy = 'egreedy' # 'egreedy' or other choices
    epsilon_min = 0.01
    decay_rate = 0.95
    batch_size = 64
    decay_epsilon = False
    
    # Learning rate
    learning_rate = 0.01
    
    # Plotting parameters
    plot = True
    
    # Nice labels for plotting
    policy_labels = {'egreedy': '$\epsilon$-greedy policy',
                  'softmax': 'Softmax policy'}                  #Something here might need to be changed

    
    ####### Experiments
    
    good_average_reward = 350 # We set this as a benchmark of good average reward reached by the algorithm
    
    Plot = LearningCurvePlot(title = 'Cartpole experiment with $\epsilon$-greedy exploration')    
    policy = 'egreedy'
    epsilons = [0.01, 1 ,0.2]
    for epsilon in epsilons:
        if epsilon == 1:
            decay_epsilon = True
        learning_curve = average_over_repetitions(policy, epsilon, smoothing_window, plot, decay_epsilon, 
                                                  n_repetitions, n_runs, learning_rate, gamma, batch_size, experience_replay)
        Plot.add_curve(learning_curve,label=r'$\epsilon$-greedy, $\epsilon $ = {}'.format(epsilon))
        decay_epsilon = False
    #policy = 'softmax'
    #temps = [0.01,0.1,1.0]
    #for temp in temps:
     #   learning_curve = average_over_repetitions(backup, n_repetitions, n_timesteps, max_episode_length, learning_rate, 
      #                                        gamma, policy, epsilon, temp, smoothing_window, plot, n)
       # Plot.add_curve(learning_curve,label=r'softmax, $ \tau $ = {}'.format(temp))
    Plot.add_hline(optimal_average_reward_per_timestep, label = '')
    Plot.save('cartpole_test.png')

if __name__ == '__main__':
    experiment()


C:\Users\nikma\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:45: DeprecationWarning: invalid escape sequence \e


Run:1, Steps:10, Epsilon:0.01
Run:2, Steps:9, Epsilon:0.01
Run:3, Steps:9, Epsilon:0.01
Run:4, Steps:11, Epsilon:0.01
Run:5, Steps:10, Epsilon:0.01
Run:6, Steps:13, Epsilon:0.01
Run:7, Steps:9, Epsilon:0.01
Run:8, Steps:10, Epsilon:0.01
Run:9, Steps:10, Epsilon:0.01
Run:10, Steps:9, Epsilon:0.01
Run:11, Steps:9, Epsilon:0.01
Run:12, Steps:12, Epsilon:0.01
Run:13, Steps:8, Epsilon:0.01
Run:14, Steps:10, Epsilon:0.01
Run:15, Steps:10, Epsilon:0.01
Run:16, Steps:9, Epsilon:0.01
Run:17, Steps:9, Epsilon:0.01
Run:18, Steps:10, Epsilon:0.01
Run:19, Steps:9, Epsilon:0.01
Run:20, Steps:9, Epsilon:0.01
Run:21, Steps:12, Epsilon:0.01
Run:22, Steps:15, Epsilon:0.01
Run:23, Steps:11, Epsilon:0.01
Run:24, Steps:9, Epsilon:0.01
Run:25, Steps:24, Epsilon:0.01
Run:26, Steps:28, Epsilon:0.01
Run:27, Steps:15, Epsilon:0.01
Run:28, Steps:9, Epsilon:0.01
Run:29, Steps:8, Epsilon:0.01
Run:30, Steps:11, Epsilon:0.01
Run:31, Steps:10, Epsilon:0.01
Run:32, Steps:11, Epsilon:0.01
Run:33, Steps:11, Epsilon:0.01

Run:93, Steps:172, Epsilon:0.01
Run:94, Steps:155, Epsilon:0.01
Run:95, Steps:151, Epsilon:0.01
Run:96, Steps:150, Epsilon:0.01
Run:97, Steps:79, Epsilon:0.01
Run:98, Steps:94, Epsilon:0.01
Run:99, Steps:102, Epsilon:0.01
Run:100, Steps:142, Epsilon:0.01
Run:1, Steps:60, Epsilon:0.01
Run:2, Steps:78, Epsilon:0.01
Run:3, Steps:72, Epsilon:0.01
Run:4, Steps:13, Epsilon:0.01
Run:5, Steps:10, Epsilon:0.01
Run:6, Steps:10, Epsilon:0.01
Run:7, Steps:10, Epsilon:0.01
Run:8, Steps:9, Epsilon:0.01
Run:9, Steps:8, Epsilon:0.01
Run:10, Steps:8, Epsilon:0.01
Run:11, Steps:10, Epsilon:0.01
Run:12, Steps:10, Epsilon:0.01
Run:13, Steps:9, Epsilon:0.01
Run:14, Steps:10, Epsilon:0.01
Run:15, Steps:10, Epsilon:0.01
Run:16, Steps:25, Epsilon:0.01
Run:17, Steps:9, Epsilon:0.01
Run:18, Steps:10, Epsilon:0.01
Run:19, Steps:8, Epsilon:0.01
Run:20, Steps:8, Epsilon:0.01
Run:21, Steps:8, Epsilon:0.01
Run:22, Steps:9, Epsilon:0.01
Run:23, Steps:10, Epsilon:0.01
Run:24, Steps:10, Epsilon:0.01
Run:25, Steps:10, E

Run:78, Steps:10, Epsilon:0.01
Run:79, Steps:9, Epsilon:0.01
Run:80, Steps:9, Epsilon:0.01
Run:81, Steps:24, Epsilon:0.01
Run:82, Steps:24, Epsilon:0.01
Run:83, Steps:27, Epsilon:0.01
Run:84, Steps:25, Epsilon:0.01
Run:85, Steps:11, Epsilon:0.01
Run:86, Steps:10, Epsilon:0.01
Run:87, Steps:9, Epsilon:0.01
Run:88, Steps:111, Epsilon:0.01
Run:89, Steps:76, Epsilon:0.01
Run:90, Steps:40, Epsilon:0.01
Run:91, Steps:40, Epsilon:0.01
Run:92, Steps:63, Epsilon:0.01
Run:93, Steps:61, Epsilon:0.01
Run:94, Steps:86, Epsilon:0.01
Run:95, Steps:123, Epsilon:0.01
Run:96, Steps:45, Epsilon:0.01
Run:97, Steps:28, Epsilon:0.01
Run:98, Steps:15, Epsilon:0.01
Run:99, Steps:14, Epsilon:0.01
Run:100, Steps:22, Epsilon:0.01
Run:1, Steps:8, Epsilon:0.01
Run:2, Steps:9, Epsilon:0.01
Run:3, Steps:10, Epsilon:0.01
Run:4, Steps:10, Epsilon:0.01
Run:5, Steps:10, Epsilon:0.01
Run:6, Steps:9, Epsilon:0.01
Run:7, Steps:9, Epsilon:0.01
Run:8, Steps:9, Epsilon:0.01
Run:9, Steps:9, Epsilon:0.01
Run:10, Steps:9, Epsilo

Run:57, Steps:361, Epsilon:0.05656162735025291
Run:58, Steps:221, Epsilon:0.053733545982740265
Run:59, Steps:191, Epsilon:0.05104686868360325
Run:60, Steps:170, Epsilon:0.04849452524942309
Run:61, Steps:185, Epsilon:0.04606979898695193
Run:62, Steps:197, Epsilon:0.04376630903760433
Run:63, Steps:167, Epsilon:0.041577993585724116
Run:64, Steps:188, Epsilon:0.03949909390643791
Run:65, Steps:186, Epsilon:0.03752413921111601
Run:66, Steps:174, Epsilon:0.03564793225056021
Run:67, Steps:191, Epsilon:0.0338655356380322
Run:68, Steps:223, Epsilon:0.032172258856130585
Run:69, Steps:155, Epsilon:0.030563645913324056
Run:70, Steps:162, Epsilon:0.029035463617657853
Run:71, Steps:158, Epsilon:0.027583690436774957
Run:72, Steps:118, Epsilon:0.02620450591493621
Run:76, Steps:266, Epsilon:0.021343733845877507
Run:80, Steps:232, Epsilon:0.017384604615803764
Run:81, Steps:349, Epsilon:0.016515374385013576
Run:88, Steps:344, Epsilon:0.011533301892006355
Run:92, Steps:214, Epsilon:0.01
Run:93, Steps:129, 

Run:68, Steps:141, Epsilon:0.032172258856130585
Run:69, Steps:170, Epsilon:0.030563645913324056
Run:70, Steps:170, Epsilon:0.029035463617657853
Run:71, Steps:163, Epsilon:0.027583690436774957
Run:72, Steps:171, Epsilon:0.02620450591493621
Run:73, Steps:271, Epsilon:0.0248942806191894
Run:74, Steps:358, Epsilon:0.023649566588229927
Run:75, Steps:198, Epsilon:0.022467088258818428
Run:76, Steps:166, Epsilon:0.021343733845877507
Run:77, Steps:171, Epsilon:0.02027654715358363
Run:78, Steps:147, Epsilon:0.019262719795904448
Run:79, Steps:171, Epsilon:0.018299583806109226
Run:80, Steps:106, Epsilon:0.017384604615803764
Run:81, Steps:131, Epsilon:0.016515374385013576
Run:82, Steps:151, Epsilon:0.015689605665762895
Run:83, Steps:133, Epsilon:0.01490512538247475
Run:84, Steps:112, Epsilon:0.014159869113351011
Run:85, Steps:139, Epsilon:0.01345187565768346
Run:86, Steps:137, Epsilon:0.012779281874799287
Run:87, Steps:165, Epsilon:0.012140317781059323
Run:88, Steps:306, Epsilon:0.01153330189200635

Run:58, Steps:24, Epsilon:0.053733545982740265
Run:59, Steps:26, Epsilon:0.05104686868360325
Run:60, Steps:27, Epsilon:0.04849452524942309
Run:61, Steps:30, Epsilon:0.04606979898695193
Run:62, Steps:24, Epsilon:0.04376630903760433
Run:63, Steps:28, Epsilon:0.041577993585724116
Run:64, Steps:64, Epsilon:0.03949909390643791
Run:65, Steps:48, Epsilon:0.03752413921111601
Run:66, Steps:101, Epsilon:0.03564793225056021
Run:67, Steps:100, Epsilon:0.0338655356380322
Run:68, Steps:123, Epsilon:0.032172258856130585
Run:69, Steps:67, Epsilon:0.030563645913324056
Run:70, Steps:78, Epsilon:0.029035463617657853
Run:71, Steps:31, Epsilon:0.027583690436774957
Run:72, Steps:59, Epsilon:0.02620450591493621
Run:73, Steps:92, Epsilon:0.0248942806191894
Run:74, Steps:50, Epsilon:0.023649566588229927
Run:75, Steps:88, Epsilon:0.022467088258818428
Run:76, Steps:74, Epsilon:0.021343733845877507
Run:77, Steps:53, Epsilon:0.02027654715358363
Run:78, Steps:53, Epsilon:0.019262719795904448
Run:79, Steps:58, Epsil

Run:72, Steps:356, Epsilon:0.02620450591493621
Run:73, Steps:131, Epsilon:0.0248942806191894
Run:74, Steps:139, Epsilon:0.023649566588229927
Run:75, Steps:163, Epsilon:0.022467088258818428
Run:76, Steps:269, Epsilon:0.021343733845877507
Run:77, Steps:171, Epsilon:0.02027654715358363
Run:78, Steps:159, Epsilon:0.019262719795904448
Run:79, Steps:171, Epsilon:0.018299583806109226
Run:80, Steps:193, Epsilon:0.017384604615803764
Run:81, Steps:154, Epsilon:0.016515374385013576
Run:82, Steps:187, Epsilon:0.015689605665762895
Run:83, Steps:172, Epsilon:0.01490512538247475
Run:84, Steps:217, Epsilon:0.014159869113351011
Run:85, Steps:202, Epsilon:0.01345187565768346
Run:86, Steps:207, Epsilon:0.012779281874799287
Run:87, Steps:230, Epsilon:0.012140317781059323
Run:88, Steps:138, Epsilon:0.011533301892006355
Run:89, Steps:180, Epsilon:0.010956636797406038
Run:90, Steps:153, Epsilon:0.010408804957535735
Run:91, Steps:162, Epsilon:0.01
Run:92, Steps:129, Epsilon:0.01
Run:96, Steps:144, Epsilon:0.0

Run:100, Steps:212, Epsilon:0.2
Run:1, Steps:8, Epsilon:0.2
Run:2, Steps:19, Epsilon:0.2
Run:3, Steps:10, Epsilon:0.2
Run:4, Steps:8, Epsilon:0.2
Run:5, Steps:10, Epsilon:0.2
Run:6, Steps:15, Epsilon:0.2
Run:7, Steps:9, Epsilon:0.2
Run:8, Steps:14, Epsilon:0.2
Run:9, Steps:10, Epsilon:0.2
Run:10, Steps:9, Epsilon:0.2
Run:11, Steps:13, Epsilon:0.2
Run:12, Steps:11, Epsilon:0.2
Run:13, Steps:10, Epsilon:0.2
Run:14, Steps:10, Epsilon:0.2
Run:15, Steps:9, Epsilon:0.2
Run:16, Steps:8, Epsilon:0.2
Run:17, Steps:9, Epsilon:0.2
Run:18, Steps:12, Epsilon:0.2
Run:19, Steps:11, Epsilon:0.2
Run:20, Steps:9, Epsilon:0.2
Run:21, Steps:12, Epsilon:0.2
Run:22, Steps:10, Epsilon:0.2
Run:23, Steps:13, Epsilon:0.2
Run:24, Steps:14, Epsilon:0.2
Run:25, Steps:11, Epsilon:0.2
Run:26, Steps:14, Epsilon:0.2
Run:27, Steps:13, Epsilon:0.2
Run:28, Steps:47, Epsilon:0.2
Run:29, Steps:25, Epsilon:0.2
Run:30, Steps:20, Epsilon:0.2
Run:31, Steps:19, Epsilon:0.2
Run:32, Steps:41, Epsilon:0.2
Run:33, Steps:77, Epsilon

Run:48, Steps:97, Epsilon:0.2
Run:49, Steps:72, Epsilon:0.2
Run:50, Steps:80, Epsilon:0.2
Run:51, Steps:58, Epsilon:0.2
Run:52, Steps:113, Epsilon:0.2
Run:53, Steps:73, Epsilon:0.2
Run:54, Steps:105, Epsilon:0.2
Run:55, Steps:167, Epsilon:0.2
Run:56, Steps:68, Epsilon:0.2
Run:57, Steps:81, Epsilon:0.2
Run:58, Steps:85, Epsilon:0.2
Run:59, Steps:60, Epsilon:0.2
Run:60, Steps:115, Epsilon:0.2
Run:61, Steps:87, Epsilon:0.2
Run:62, Steps:73, Epsilon:0.2
Run:63, Steps:103, Epsilon:0.2
Run:64, Steps:209, Epsilon:0.2
Run:65, Steps:301, Epsilon:0.2
Run:66, Steps:206, Epsilon:0.2
Run:67, Steps:173, Epsilon:0.2
Run:68, Steps:206, Epsilon:0.2
Run:69, Steps:252, Epsilon:0.2
Run:70, Steps:217, Epsilon:0.2
Run:71, Steps:399, Epsilon:0.2
Run:1, Steps:9, Epsilon:0.2
Run:2, Steps:10, Epsilon:0.2
Run:3, Steps:10, Epsilon:0.2
Run:4, Steps:12, Epsilon:0.2
Run:5, Steps:17, Epsilon:0.2
Run:6, Steps:10, Epsilon:0.2
Run:7, Steps:24, Epsilon:0.2
Run:8, Steps:13, Epsilon:0.2
Run:9, Steps:13, Epsilon:0.2
Run:10,